<a href="https://colab.research.google.com/github/patiladesh523/Adesh-Projects/blob/main/Fake_News_Detection_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fake News Detection System

1) Load Dataset

In [ ]:
import pandas as pd
# load dataset
true_df = pd.read_csv("/content/drive/MyDrive/Colab Dataset/True.csv")
true_df.head(5)

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [ ]:
fake_df = pd.read_csv("/content/drive/MyDrive/Colab Dataset/Fake.csv")
fake_df.head(5)

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [ ]:
# Add Labels
true_df['label'] = 1 # Real
fake_df['label'] = 0 # Fake

In [ ]:
# Combine datasets
df = pd.concat([true_df,fake_df],ignore_index=True)
df = df[['text','label']] # use only text and label

In [ ]:
df.head(5)

,text,label
0,WASHINGTON (Reuters) - The head of a conservat...,1
1,WASHINGTON (Reuters) - Transgender people will...,1
2,WASHINGTON (Reuters) - The special counsel inv...,1
3,WASHINGTON (Reuters) - Trump campaign adviser ...,1
4,SEATTLE/WASHINGTON (Reuters) - President Donal...,1


In [ ]:
df.tail(5)

,text,label
44893,21st Century Wire says As 21WIRE reported earl...,0
44894,21st Century Wire says It s a familiar theme. ...,0
44895,Patrick Henningsen 21st Century WireRemember ...,0
44896,21st Century Wire says Al Jazeera America will...,0
44897,21st Century Wire says As 21WIRE predicted in ...,0


2) Text Cleaning Function(NLP)

In [ ]:
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
stop_words = set(stopwords.words('english'))

In [ ]:
# lower case
def clean_text(text):
  text = text.lower()
  # remove URLs
  text = re.sub(r"http\S+|www\S+|https\S+",'',text)
  # remove punctuation / number
  text = re.sub(r'[^A-Za-z\s]','',text)
  # remove stopwords
  text = ''.join([word for word in text.split() if word not in stop_words])
  return text
df['clean_text'] = df['text'].apply(clean_text)


3) TF-IDF Vectorization

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
tfidf = TfidfVectorizer(max_features=5000)
X = tfidf.fit_transform(df['clean_text'])
y = df['label']

4) Train-Test Split

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train , X_test , y_train , y_test = train_test_split(X,y,test_size=0.2,random_state=42)

5) Train Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
model = LogisticRegression()
model.fit(X_train,y_train)

LogisticRegression()

6) Model Evaluation

In [ ]:
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
print("Accuracy:",accuracy_score(y_test,y_pred))

Accuracy: 0.6691536748329622


In [ ]:
print("Confusion Matrix:\n",confusion_matrix(y_test,y_pred))

Confusion Matrix:
 [[1679 2971]
 [   0 4330]]


In [ ]:
print("Classification Report:",classification_report(y_test,y_pred))

Classification Report:               precision    recall  f1-score   support

           0       1.00      0.36      0.53      4650
           1       0.59      1.00      0.74      4330

    accuracy                           0.67      8980
   macro avg       0.80      0.68      0.64      8980
weighted avg       0.80      0.67      0.63      8980



7) Predict on New News Article

In [ ]:
def predict_news(news_text):
  cleaned = clean_text(news_text)
  vector = tfidf.transform([cleaned])
  prediction = model.predict(vector)
  return "REAL" if prediction[0] == 1 else "FAKE"


In [ ]:
# Example
print(predict_news("The Prime Minister addressed the nation on economic policies"))

REAL
